In [17]:
import os

import pandas as pd
import numpy as np

import plotly.graph_objects as go

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.utils import resample

import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor

import optuna

# Define Paths and Load Data

In [18]:
data_folder = os.path.join("..", "..", "..", "data", "berlin")
clean_data_folder = os.path.join(data_folder, "clean_data")

In [19]:
surface_df = pd.read_excel(os.path.join(clean_data_folder, "surface.xlsx"))

In [20]:
ground_df = pd.read_excel(os.path.join(clean_data_folder, "ground.xlsx"))

In [21]:
diff_columns = ["DateTime", "Station"]
bacteria_columns = [
    "E.Coli (MPN/100ml)",
    "Enterococcus (MPN/100ml)",
    "Coliform (MPN/100ml)"
]

# Modelling

In [22]:
def extend_features(df: pd.DataFrame, lags: int, rolling_window: int, poly_degree: int):
    
    initial_features = df.columns
    # add polynomial features
    poly = PolynomialFeatures(degree=poly_degree)
    df_poly = poly.fit_transform(df)
    df = pd.DataFrame(df_poly, columns=poly.get_feature_names_out(df.columns))
    
    # add lagged, rolling and expanding features for each variable in df
    for col in initial_features.difference(["Year", "Month"]):
        for lag in range(1, lags + 1):
            df[f"{col}_lag{lag}"] = df[col].shift(lag)
            
        df[f"{col}_rolling{rolling_window}"] = df[col].rolling(rolling_window).mean()
        
    # fill NaN values with bfill
    df.bfill(inplace=True)
    
    df.drop(columns=['1'], inplace=True)
    return df

## Surface

In [23]:
drop_columns = [
    "Ammonium (mg/l)",
    "Conductivity (µS/cm)",
    "Dissolved Oxygen (mg/l)",
    "Nitrate (mg/l)",
    "pH",
]

In [24]:
surface_df.drop(columns=drop_columns, inplace=True)

In [25]:
train_size = 0.7

In [26]:
datasets = {}

# Prepare the data for the models
for station_id in surface_df['Station'].unique():
    df = surface_df[surface_df['Station'] == station_id]
    
    # add the year and month columns
    df["Year"] = df["DateTime"].dt.year
    df["Month"] = df["DateTime"].dt.month
    
    # Save the datetime column for later (drop diff returns error
    # if I remove it before)
    datetime_column = df.drop(columns=bacteria_columns).dropna()["DateTime"]
    
    df = df.drop(columns=diff_columns + bacteria_columns).dropna()
    
    X = df.drop(columns=["DOC (mg/l)"])
    y = df[["DOC (mg/l)"]]
    
    X = extend_features(X, lags=1, rolling_window=3, poly_degree=2)
    
    # Normalize the data
    scaler = MinMaxScaler()
    cols = X.columns
    
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X, columns=cols)
    
    # Add the datetime column back
    X["DateTime"] = datetime_column.values
    y["DateTime"] = datetime_column.values
    
    
    X = X.set_index("DateTime")
    y = y.set_index("DateTime")
    
    X_tr, X_ts = X[:int(train_size * len(X))], X[int(train_size * len(X)):]
    y_tr, y_ts = y[:int(train_size * len(y))], y[int(train_size * len(y)):]
    
    datasets[station_id] = (X_tr, X_ts, y_tr, y_ts)

/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_85690/3521460032.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"] = df["DateTime"].dt.year
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_85690/3521460032.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Month"] = df["DateTime"].dt.month
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_85690/3521460032.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

### Linear Regression

In [27]:
lr_results = {}

for station_id in surface_df['Station'].unique():
    X_tr, X_ts, y_tr, y_ts = datasets[station_id]
    
    model = sm.OLS(y_tr, sm.add_constant(X_tr))
    results = model.fit()
    
    predictions = results.get_prediction(sm.add_constant(X_ts)).summary_frame(alpha=0.05)
    
    lr_results[station_id] = {
        "y_pred": predictions['mean'],
        "y_pred_lower": predictions['mean_ci_lower'],
        "y_pred_upper": predictions['mean_ci_upper'],
        "model": results,
        "rmse": np.sqrt(mean_squared_error(y_ts, predictions["mean"])),
        "r2": r2_score(y_ts, predictions["mean"]),
    }

### Random Forest

#### Hyperparameter Tuning

In [28]:
def fit_and_validate_rf(
    X,
    y,
    train_index,
    val_index,
    params,
):
    X_tr, X_val = X.iloc[train_index, :], X.iloc[val_index, :]
    y_tr, y_val = y.iloc[train_index], y.iloc[val_index]

    model = RandomForestRegressor(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        min_samples_split=params["min_samples_split"],
        min_samples_leaf=params["min_samples_leaf"],
        random_state=42,
    )

    # train model
    _ = model.fit(X_tr, y_tr)

    # obtain predictions
    y_val_pred = model.predict(X_val)

    # return metrics
    return mean_squared_error(y_val, y_val_pred, squared=False)

In [29]:
def objective(trial: optuna.trial.Trial, X_cv, y_cv) -> float:
    # define the hyperparameters to search over
    
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 500, step=10),
        "max_depth": trial.suggest_int("max_depth", 1, 32),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
    }
    
    
    n_splits = 5
    cv = TimeSeriesSplit(n_splits=n_splits)
    cv_rmse = [None] * n_splits
    for i, (train_index, test_index) in enumerate(cv.split(X_cv)):
        cv_rmse[i] = fit_and_validate_rf(
            X_cv, y_cv, train_index, test_index, params
        )

    # saving the individual fold holdout metrics
    # uncomment this line if you don't want this
    trial.set_user_attr("split_rmse", cv_rmse)

    return np.mean(cv_rmse)

In [30]:
rf_studies = {}

for station_id in surface_df['Station'].unique():
    
    X_tr, _, y_tr, _ = datasets[station_id]
    
    if os.path.exists(f"RandomForest-Station{station_id}-Extended.sqlite3"):
        
        study = optuna.load_study(
        study_name="Hyperparameter Tuning - RandomForest"
        + " + "
        + f"Station {station_id}",
        storage=f"sqlite:///RandomForest-Station{station_id}-Extended.sqlite3",
    )

    else:
        
        study = optuna.create_study(
            direction="minimize",
            storage=f"sqlite:///RandomForest-Station{station_id}-Extended.sqlite3",
            study_name="Hyperparameter Tuning - RandomForest"
            + " + "
            + f"Station {station_id}",
            load_if_exists=True,
        )
        study.optimize(lambda trial: objective(trial, X_tr, y_tr), n_trials=100, show_progress_bar=True)
    
    rf_studies[station_id] = study

#### Prediction

In [31]:
rf_results = {}

n_iterations = 100

for station_id in surface_df['Station'].unique():
    params = rf_studies[station_id].best_params
    
    X_tr, X_ts, y_tr, y_ts = datasets[station_id]
    
    n_size = len(X_tr)
    predictions = np.zeros((len(X_ts), n_iterations))
    metrics = []
    
    for i in range(n_iterations):
    # Bootstrap sample (random state changes each iteration)
        X_resampled, y_resampled = resample(X_tr, y_tr, n_samples=n_size, random_state=i)
        
        # Train the model with the best hyperparameters
        model = RandomForestRegressor(random_state=42, **params)
        model.fit(X_resampled, y_resampled)
        
        # Predict on the validation set
        y_pred = model.predict(X_ts)
        predictions[:, i] = y_pred
        
        # Calculate and store the metric (e.g., RMSE)
        metric = mean_squared_error(y_ts, y_pred, squared=False)
        metrics.append(metric)
        
    
    
    # Convert to a numpy array for easier calculation
    metrics = np.array(metrics)
    
    # Calculate the mean RMSE
    mean_rmse = np.mean(metrics)
    
    # Calculate 95% confidence interval of the predictions
    lower_bound = np.percentile(predictions, 2.5, axis=1)
    upper_bound = np.percentile(predictions, 97.5, axis=1)
    
    # Calculate the mean predictions
    mean_predictions = np.mean(predictions, axis=1)
    
    rf_results[station_id] = {
        "y_pred": mean_predictions,
        "y_pred_lower": lower_bound,
        "y_pred_upper": upper_bound,
        "model": model,
        "rmse": mean_rmse,
        "r2": r2_score(y_ts, mean_predictions),
    }

/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_

### XGBoost

#### Hyperparameter Tuning

In [32]:
def fit_and_validate_xgb_model(
    X,
    y,
    train_index,
    val_index,
    params,
):
    X_tr, X_val = X.iloc[train_index, :], X.iloc[val_index, :]
    y_tr, y_val = y.iloc[train_index], y.iloc[val_index]

    model = xgb.XGBRegressor(random_state=42, **params)

    # train model
    _ = model.fit(X_tr, y_tr)

    # obtain predictions
    y_val_pred = model.predict(X_val)

    # return metrics
    return mean_squared_error(y_val, y_val_pred, squared=False)

In [33]:
def objective(trial: optuna.trial.Trial, X_cv, y_cv) -> float:
    eta = trial.suggest_float("eta", 1e-5, 1, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-8, 1, log=True)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-8, 1, log=True)
    learning_rate = trial.suggest_float(
        "learning_rate", 1e-5, 1, log=True
    )
    n_estimators = trial.suggest_int("n_estimators", 1, 500)
    updater = trial.suggest_categorical(
        "updater", ["shotgun", "coord_descent"]
    )

    params = {
        "objective": "reg:squarederror",
        "booster": "gblinear",
        "eta": eta,
        "reg_lambda": reg_lambda,
        "reg_alpha": reg_alpha,
        "learning_rate": learning_rate,
        "updater": updater,
        "n_estimators": n_estimators,
        "eval_metric": "rmse",
    }

    n_splits = 5
    cv = TimeSeriesSplit(n_splits=n_splits)
    cv_rmse = [None] * n_splits
    for i, (train_index, test_index) in enumerate(
        cv.split(X_cv, y_cv)
    ):
        cv_rmse[i] = fit_and_validate_xgb_model(
            X_cv,
            y_cv,
            train_index,
            test_index,
            params,
        )

    # saving the individual fold holdout metrics
    # uncomment this line if you don't want this
    trial.set_user_attr("split_rmse", cv_rmse)

    return np.mean(cv_rmse)

In [34]:
xgb_studies = {}

for station_id in surface_df['Station'].unique():
    
    X_tr, _, y_tr, _ = datasets[station_id]

    if os.path.exists(f"XGBoost-Station{station_id}-Extended.sqlite3"):
            
        study = optuna.load_study(
        study_name="Hyperparameter Tuning - XGBoost"
        + " + "
        + f"Station{station_id}",
        storage=f"sqlite:///XGBoost-Station{station_id}-Extended.sqlite3",
        )
            
    else:
            
        study = optuna.create_study(
            direction="minimize",
            storage=f"sqlite:///XGBoost-Station{station_id}-Extended.sqlite3",
            study_name="Hyperparameter Tuning - XGBoost"
            + " + "
            + f"Station{station_id}",
            load_if_exists=True,
        )
        study.optimize(lambda trial: objective(trial, X_tr, y_tr), n_trials=100, show_progress_bar=True)
            
    xgb_studies[station_id] = study

#### Prediction

In [35]:
xgb_results = {}

n_iterations = 100

for station_id in surface_df['Station'].unique():
    params = xgb_studies[station_id].best_params
    
    params["objective"] = "reg:squarederror"
    params["booster"] = "gblinear"
    
    X_tr, X_ts, y_tr, y_ts = datasets[station_id]
    
    n_size = len(X_tr)
    predictions = np.zeros((len(X_ts), n_iterations))
    metrics = []
    
    for i in range(n_iterations):
    # Bootstrap sample (random state changes each iteration)
        X_resampled, y_resampled = resample(X_tr, y_tr, n_samples=n_size, random_state=i)
        
        # Train the model with the best hyperparameters
        model = xgb.XGBRegressor(**params, random_state=42)
        model.fit(X_resampled, y_resampled)
        
        # Predict on the validation set
        y_pred = model.predict(X_ts)
        predictions[:, i] = y_pred
        
        # Calculate and store the metric (e.g., RMSE)
        metric = mean_squared_error(y_ts, y_pred, squared=False)
        metrics.append(metric)
        
    
    
    # Convert to a numpy array for easier calculation
    metrics = np.array(metrics)
    
    # Calculate the mean RMSE
    mean_rmse = np.mean(metrics)
    
    # Calculate 95% confidence interval of the predictions
    lower_bound = np.percentile(predictions, 2.5, axis=1)
    upper_bound = np.percentile(predictions, 97.5, axis=1)
    
    # Calculate the mean predictions
    mean_predictions = np.mean(predictions, axis=1)
    
    xgb_results[station_id] = {
        "y_pred": mean_predictions,
        "y_pred_lower": lower_bound,
        "y_pred_upper": upper_bound,
        "model": model,
        "rmse": mean_rmse,
        "r2": r2_score(y_ts, mean_predictions),
    }

### LightGBM

#### Hyperparameter Tuning

In [36]:
def fit_and_validate_lgbm_model(
    X,
    y,
    train_index,
    val_index,
    params,
):
    X_tr, X_val = X.iloc[train_index, :], X.iloc[val_index, :]
    y_tr, y_val = y.iloc[train_index], y.iloc[val_index]

    model = LGBMRegressor(
        objective="regression",
        random_state=42,
        linear_tree=True,
    )

    if params is not None:
        model.set_params(**params)

    # train model
    _ = model.fit(X_tr, y_tr)

    # obtain predictions
    y_val_pred = model.predict(X_val)

    # return metrics
    return mean_squared_error(y_val, y_val_pred, squared=False)

In [37]:
def objective(trial: optuna.trial.Trial, X_cv, y_cv) -> float:
    config = {
        "n_estimators": trial.suggest_int(
            "n_estimators", 1, 20, step=1
        ),
        "learning_rate": trial.suggest_float(
            "learning_rate", 1e-3, 1, log=True
        ),
        "max_depth": trial.suggest_int("max_depth", 2, 16, step=1),
        "num_leaves": trial.suggest_int("num_leaves", 2, 20, step=1),
        "min_data_in_leaf": trial.suggest_int(
            "min_data_in_leaf", 2, 50, step=1
        ),
        "lambda_l1": trial.suggest_float(
            "lambda_l1", 1e-3, 10, log=True
        ),
        "lambda_l2": trial.suggest_float(
            "lambda_l2", 1e-3, 10, log=True
        ),
        "min_split_gain": trial.suggest_float(
            "min_split_gain", 0, 15, step=0.5
        ),
        "subsample": trial.suggest_float("subsample", 0.1, 1),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 1e-3, 1, log=True
        ),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 1e-3, 1, log=True
        ),
        "min_child_samples": trial.suggest_int(
            "min_child_samples", 20, 1000, log=True
        ),
        "max_bin": trial.suggest_int("max_bin", 10, 500, step=10),
    }

    n_splits = 5
    cv = TimeSeriesSplit(n_splits=n_splits)
    cv_rmse = [None] * n_splits
    for i, (train_index, test_index) in enumerate(
        cv.split(X_cv, y_cv)
    ):
        cv_rmse[i] = fit_and_validate_lgbm_model(
            X_cv,
            y_cv,
            train_index,
            test_index,
            config,
        )
        
    # saving the individual fold holdout metrics
    # uncomment this line if you don't want this
    trial.set_user_attr("split_rmse", cv_rmse)
    
    return np.mean(cv_rmse)

In [38]:
lgbm_studies = {}

for station_id in surface_df['Station'].unique():
        
        X_tr, _, y_tr, _ = datasets[station_id]
    
        if os.path.exists(f"LGBM-Station{station_id}-Extended.sqlite3"):
                
            study = optuna.load_study(
            study_name="Hyperparameter Tuning - LGBM"
            + " + "
            + f"Station {station_id}",
            storage=f"sqlite:///LGBM-Station{station_id}-Extended.sqlite3",
            )
                
        else:
                
            study = optuna.create_study(
                direction="minimize",
                storage=f"sqlite:///LGBM-Station{station_id}-Extended.sqlite3",
                study_name="Hyperparameter Tuning - LGBM"
                + " + "
                + f"Station {station_id}",
                load_if_exists=True,
            )
            study.optimize(lambda trial: objective(trial, X_tr, y_tr), n_trials=100, show_progress_bar=True)
                
        lgbm_studies[station_id] = study

#### Prediction

In [39]:
lgbm_results = {}

n_iterations = 100

for station_id in surface_df['Station'].unique():
    params = lgbm_studies[station_id].best_params
    
    X_tr, X_ts, y_tr, y_ts = datasets[station_id]
    
    n_size = len(X_tr)
    predictions = np.zeros((len(X_ts), n_iterations))
    metrics = []
    
    for i in range(n_iterations):
    # Bootstrap sample (random state changes each iteration)
        X_resampled, y_resampled = resample(X_tr, y_tr, n_samples=n_size, random_state=i)
        
        # Train the model with the best hyperparameters
        model = LGBMRegressor(
        objective="regression",
        random_state=42,
        linear_tree=True,
        )
        
        model.set_params(**params)
        
        model.fit(X_resampled, y_resampled)
        
        # Predict on the validation set
        y_pred = model.predict(X_ts)
        predictions[:, i] = y_pred
        
        # Calculate and store the metric (e.g., RMSE)
        metric = mean_squared_error(y_ts, y_pred, squared=False)
        metrics.append(metric)
        
    
    
    # Convert to a numpy array for easier calculation
    metrics = np.array(metrics)
    
    # Calculate the mean RMSE
    mean_rmse = np.mean(metrics)
    
    # Calculate 95% confidence interval of the predictions
    lower_bound = np.percentile(predictions, 2.5, axis=1)
    upper_bound = np.percentile(predictions, 97.5, axis=1)
    
    # Calculate the mean predictions
    mean_predictions = np.mean(predictions, axis=1)
    
    lgbm_results[station_id] = {
        "y_pred": mean_predictions,
        "y_pred_lower": lower_bound,
        "y_pred_upper": upper_bound,
        "model": model,
        "rmse": mean_rmse,
        "r2": r2_score(y_ts, mean_predictions),
    }

[LightGBM] [Warning] lambda_l1 is set=0.008681257378218206, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.008681257378218206
[LightGBM] [Warning] lambda_l2 is set=0.0035375709277243434, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0035375709277243434
[LightGBM] [Warning] bagging_fraction is set=0.6067749848195905, subsample=0.9444290735841788 will be ignored. Current value: bagging_fraction=0.6067749848195905
[LightGBM] [Warning] min_data_in_leaf is set=22, min_child_samples=300 will be ignored. Current value: min_data_in_leaf=22
[LightGBM] [Warning] feature_fraction is set=0.011827151587395912, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.011827151587395912
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] lambda_l1 is set=0.008681257378218206, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.008681257378218206
[LightGBM] [Warning] lambda_l2 is set=0.0035375709277243434, re

### MultiLayerPerceptron

#### Hyperparameter Tuning

In [40]:
def fit_and_validate_nn_model(
    X,
    y,
    train_index,
    val_index,
    params,
):
    X_tr, X_val = X.iloc[train_index, :], X.iloc[val_index, :]
    y_tr, y_val = y.iloc[train_index], y.iloc[val_index]

    model = MLPRegressor(
        random_state=42,
        hidden_layer_sizes=tuple(params["layers"]),
        max_iter=1000,
    )

    param = params.copy()
    param.pop("layers")
    model.set_params(**param)

    # train model
    _ = model.fit(X_tr, y_tr.values.ravel())

    # obtain predictions
    y_val_pred = model.predict(X_val)

    # return metrics
    return mean_squared_error(y_val, y_val_pred, squared=False)

In [41]:
def objective(trial: optuna.trial.Trial, X_cv, y_cv) -> float:
    config = {
        "layers": [
            trial.suggest_int(f"n_units_{i}", 50, 100, step=5)
            for i in range(trial.suggest_int("n_layers", 2, 2))
        ],
        "activation": trial.suggest_categorical(
            "activation", ["identity", "logistic", "tanh", "relu"]
        ),
        "solver": trial.suggest_categorical("solver", ["sgd", "adam"]),
        "alpha": trial.suggest_float("alpha", 1e-5, 1),
        "learning_rate": trial.suggest_categorical(
            "learning_rate", ["constant", "invscaling", "adaptive"]
        ),
        "power_t": trial.suggest_float("power_t", 0.1, 1),
        "beta_1": trial.suggest_float("beta_1", 0.1, 1),
        "beta_2": trial.suggest_float("beta_2", 0.1, 1),
        "epsilon": trial.suggest_float("epsilon", 1e-8, 1),
        "early_stopping": True,
    }

    n_splits = 5
    cv = TimeSeriesSplit(n_splits=n_splits)
    cv_rmse = [None] * n_splits
    for i, (train_index, test_index) in enumerate(
        cv.split(X_cv, y_cv)
    ):
        cv_rmse[i] = fit_and_validate_nn_model(
            X_cv,
            y_cv,
            train_index,
            test_index,
            config,
        )
        
    # saving the individual fold holdout metrics
    # uncomment this line if you don't want this
    trial.set_user_attr("split_rmse", cv_rmse)
    
    return np.mean(cv_rmse)

In [42]:
mlp_studies = {}

for station_id in surface_df['Station'].unique():
            
    X_tr, _, y_tr, _ = datasets[station_id]

    if os.path.exists(f"MLP-Station{station_id}-Extended.sqlite3"):
            
        study = optuna.load_study(
        study_name="Hyperparameter Tuning - MLP"
        + " + "
        + f"Station {station_id}",
        storage=f"sqlite:///MLP-Station{station_id}-Extended.sqlite3",
        )
            
    else:
            
        study = optuna.create_study(
            direction="minimize",
            storage=f"sqlite:///MLP-Station{station_id}-Extended.sqlite3",
            study_name="Hyperparameter Tuning - MLP"
            + " + "
            + f"Station {station_id}",
            load_if_exists=True,
        )
        study.optimize(lambda trial: objective(trial, X_tr, y_tr), n_trials=100, show_progress_bar=True)
            
    mlp_studies[station_id] = study

[I 2024-08-11 13:12:41,692] A new study created in RDB with name: Hyperparameter Tuning - MLP + Station 305


  0%|          | 0/100 [00:00<?, ?it/s]

/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-ch

[I 2024-08-11 13:12:43,676] Trial 0 finished with value: 1.1514721960078518 and parameters: {'n_layers': 2, 'n_units_0': 60, 'n_units_1': 50, 'activation': 'identity', 'solver': 'adam', 'alpha': 0.7089248938819706, 'learning_rate': 'constant', 'power_t': 0.5364303567609794, 'beta_1': 0.4468611797258778, 'beta_2': 0.9755537413482973, 'epsilon': 0.8596652865112296}. Best is trial 0 with value: 1.1514721960078518.


/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-ch

...

[I 2024-08-11 13:20:03,122] Trial 94 finished with value: 1.1490963665801883 and parameters: {'n_layers': 2, 'n_units_0': 60, 'n_units_1': 85, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.88186364375657, 'learning_rate': 'invscaling', 'power_t': 0.6289648904172233, 'beta_1': 0.21810821213838766, 'beta_2': 0.6694425967766797, 'epsilon': 0.2184159115691794}. Best is trial 83 with value: 0.9145953038244417.


/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[I 2024-08-11 13:20:05,702] Trial 95 finished with value: 0.9480584393047877 and parameters: {'n_layers': 2, 'n_units_0': 65, 'n_units_1': 80, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.9172657921061301, 'learning_rate': 'invscaling', 'power_t': 0.6771009314983071, 'beta_1': 0.4928053037847258, 'beta_2': 0.7005522483797156, 'epsilon': 0.14228101924409517}. Best is trial 83 with value: 0.9145953038244417.
[I 2024-08-11 13:20:06,411] Trial 96 finished with value: 1.014393483013144 and parameters: {'n_layers': 2, 'n_units_0': 70, 'n_units_1': 75, 'activation': 'logistic', 'solver': 'adam', 'alpha': 0.8718846099698805, 'learning_rate': 'adaptive', 'power_t': 0.6774552118840761, 'beta_1': 0.5351277337027204, 'beta_2': 0.7029855648850276, 'epsilon': 0.05681179750570656}. Best is trial 83 with value: 0.9145953038244417.
[I 2024-08-11 13:20:06,622] Trial 97 finished with value: 2.032150635360703 and parameters: {'n_layers': 2, 'n_units_0': 60, 'n_units_1': 80, 'activation': 'identity',

/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[I 2024-08-11 13:20:09,474] Trial 98 finished with value: 0.9177201853869097 and parameters: {'n_layers': 2, 'n_units_0': 65, 'n_units_1': 70, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.32072325785724054, 'learning_rate': 'invscaling', 'power_t': 0.7479061068377202, 'beta_1': 0.48728054335446636, 'beta_2': 0.7341307278915065, 'epsilon': 0.12525876481351822}. Best is trial 83 with value: 0.9145953038244417.
[I 2024-08-11 13:20:11,769] Trial 99 finished with value: 1.051738033778396 and parameters: {'n_layers': 2, 'n_units_0': 65, 'n_units_1': 65, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.3398223861707585, 'learning_rate': 'invscaling', 'power_t': 0.341392323852658, 'beta_1': 0.4826465524377923, 'beta_2': 0.7360200100163135, 'epsilon': 0.1210461211393488}. Best is trial 83 with value: 0.9145953038244417.


#### Prediction

In [43]:
mlp_results = {}

n_iterations = 100

for station_id in surface_df['Station'].unique():
    params = mlp_studies[station_id].best_params
    
    X_tr, X_ts, y_tr, y_ts = datasets[station_id]
    
    n_size = len(X_tr)
    predictions = np.zeros((len(X_ts), n_iterations))
    metrics = []
    
    for i in range(n_iterations):
        
        # Copy since we will be modifying the params
        params_copy = params.copy()
        
        # Bootstrap sample (random state changes each iteration)
        X_resampled, y_resampled = resample(X_tr, y_tr, n_samples=n_size, random_state=i)
        
        
        hidden_layer_sizes = [
            params_copy[f"n_units_{k}"] for k in range(params_copy["n_layers"])
        ]

        for j in range(params_copy["n_layers"]):
            params_copy.pop(f"n_units_{j}")

        params_copy.pop("n_layers")
            
        model = MLPRegressor(
            random_state=42,
            hidden_layer_sizes=hidden_layer_sizes,
            max_iter=1000,
        )
    
        model.set_params(**params_copy)
        
        # Predict on the validation set
        model.fit(X_resampled, y_resampled.values.ravel())
        y_pred = model.predict(X_ts)
        predictions[:, i] = y_pred
        
        # Calculate and store the metric (e.g., RMSE)
        metric = mean_squared_error(y_ts, y_pred, squared=False)
        metrics.append(metric)
        
    
    
    # Convert to a numpy array for easier calculation
    metrics = np.array(metrics)
    
    # Calculate the mean RMSE
    mean_rmse = np.mean(metrics)
    
    # Calculate 95% confidence interval of the predictions
    lower_bound = np.percentile(predictions, 2.5, axis=1)
    upper_bound = np.percentile(predictions, 97.5, axis=1)
    
    # Calculate the mean predictions
    mean_predictions = np.mean(predictions, axis=1)
    
    mlp_results[station_id] = {
        "y_pred": mean_predictions,
        "y_pred_lower": lower_bound,
        "y_pred_upper": upper_bound,
        "model": model,
        "rmse": mean_rmse,
        "r2": r2_score(y_ts, mean_predictions),
    }

/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-change-MEYtuKH4-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/massimilianoarca/Library/Caches/pypoetry/virtualenvs/climate-ch

### Plot

In [44]:
# plot the results

for station_id in surface_df['Station'].unique():
    print(f"=== Station {station_id} ===")
    
    X_tr, X_ts, y_tr, y_ts = datasets[station_id]
    
    lr_result = lr_results[station_id]
    rf_result = rf_results[station_id]
    xgb_result = xgb_results[station_id]
    lgbm_result = lgbm_results[station_id]
    mlp_result = mlp_results[station_id]
    
    rmse_lr = lr_result["rmse"]
    rmse_rf = rf_result["rmse"]
    rmse_xgb = xgb_result["rmse"]
    rmse_lgbm = lgbm_result["rmse"]
    rmse_mlp = mlp_result["rmse"]
    
    r2_lr = lr_result["r2"]
    r2_rf = rf_result["r2"]
    r2_xgb = xgb_result["r2"]
    r2_lgbm = lgbm_result["r2"]
    r2_mlp = mlp_result["r2"]
    
    print(f"Linear Regression RMSE: {rmse_lr}")
    print(f"Random Forest RMSE: {rmse_rf}")
    print(f"XGBoost RMSE: {rmse_xgb}")
    print(f"LightGBM RMSE: {rmse_lgbm}")
    print(f"Neural Network RMSE: {rmse_mlp}")
    
    print()
    
    print(f"Linear Regression R2: {r2_lr}")
    print(f"Random Forest R2: {r2_rf}")
    print(f"XGBoost R2: {r2_xgb}")
    print(f"LightGBM R2: {r2_lgbm}")
    print(f"Neural Network R2: {r2_mlp}")
    
    
    
    fig = go.Figure()
    
    # TRUE
    
    # add both the training and testing data in a unique trace
    y_true = pd.concat([y_tr, y_ts])
    
    fig.add_trace(
        go.Scatter(
            x=y_true.index,
            y=y_true["DOC (mg/l)"],
            mode="lines",
            name="True",
            line=dict(color="black"),
        )
    )
    
    # LINEAR REGRESSION
    
    fig.add_trace(
        go.Scatter(
            x=y_ts.index,
            y=lr_result["y_pred"],
            mode="lines",
            name="Linear Regression",
            line=dict(color="blue"),
        )
    )
    
    # add confidence intervals
    fig.add_traces(
        [
            go.Scatter(
                x=y_ts.index,
                y=lr_result["y_pred_lower"],
                mode="lines",
                line_color="blue",
                line=dict(dash="dash"),
                showlegend=False,
            ),
            go.Scatter(
                x=y_ts.index,
                y=lr_result["y_pred_upper"],
                mode="lines",
                line_color="blue",
                line=dict(dash="dash"),
                name="95% CI",
                fill="tonexty",
                fillcolor="rgba(0,0,255,0.2)"
            )
        ]
    )
    
    # RANDOM FOREST
    
    fig.add_trace(
        go.Scatter(
            x=y_ts.index,
            y=rf_result["y_pred"],
            mode="lines",
            name="Random Forest",
            line=dict(color="red"),
        )
    )
    
    # add confidence intervals
    fig.add_traces(
        [
            go.Scatter(
                x=y_ts.index,
                y=rf_result["y_pred_lower"],
                mode="lines",
                line_color="red",
                line=dict(dash="dash"),
                showlegend=False,
            ),
            go.Scatter(
                x=y_ts.index,
                y=rf_result["y_pred_upper"],
                mode="lines",
                line_color="red",
                line=dict(dash="dash"),
                name="95% CI",
                fill="tonexty",
                fillcolor="rgba(255,0,0,0.2)"
            )
        ]
    )
    
    # XGBOOST
    
    fig.add_trace(
        go.Scatter(
            x=y_ts.index,
            y=xgb_result["y_pred"],
            mode="lines",
            name="XGBoost",
            line=dict(color="green"),
        )
    )
    
    # add confidence intervals
    fig.add_traces(
        [
            go.Scatter(
                x=y_ts.index,
                y=xgb_result["y_pred_lower"],
                mode="lines",
                line_color="green",
                line=dict(dash="dash"),
                showlegend=False,
            ),
            go.Scatter(
                x=y_ts.index,
                y=xgb_result["y_pred_upper"],
                mode="lines",
                line_color="green",
                line=dict(dash="dash"),
                name="95% CI",
                fill="tonexty",
                fillcolor="rgba(0,255,0,0.2)"
            )
        ]
    )
    
    # LGBM
    
    fig.add_trace(
        go.Scatter(
            x=y_ts.index,
            y=lgbm_result["y_pred"],
            mode="lines",
            name="LightGBM",
            line=dict(color="purple"),
        )
    )
    
    # add confidence intervals
    fig.add_traces(
        [
            go.Scatter(
                x=y_ts.index,
                y=lgbm_result["y_pred_lower"],
                mode="lines",
                line_color="purple",
                line=dict(dash="dash"),
                showlegend=False,
            ),
            go.Scatter(
                x=y_ts.index,
                y=lgbm_result["y_pred_upper"],
                mode="lines",
                line_color="purple",
                line=dict(dash="dash"),
                name="95% CI",
                fill="tonexty",
                fillcolor="rgba(128,0,128,0.2)"
            )
        ]
    )
    
    # MLP
    
    fig.add_trace(
        go.Scatter(
            x=y_ts.index,
            y=mlp_result["y_pred"],
            mode="lines",
            name="Neural Network",
            line=dict(color="orange"),
        )
    )
    
    # add confidence intervals
    fig.add_traces(
        [
            go.Scatter(
                x=y_ts.index,
                y=mlp_result["y_pred_lower"],
                mode="lines",
                line_color="orange",
                line=dict(dash="dash"),
                showlegend=False,
            ),
            go.Scatter(
                x=y_ts.index,
                y=mlp_result["y_pred_upper"],
                mode="lines",
                line_color="orange",
                line=dict(dash="dash"),
                name="95% CI",
                fill="tonexty",
                fillcolor="rgba(255,165,0,0.2)"
            )
        ]
    )
    
    fig.update_layout(
        title=f"DOC (mg/l) - Station {station_id}",
        xaxis_title="Date",
        yaxis_title="DOC (mg/l)",
    )
    
    fig.show()

=== Station 105 ===
Linear Regression RMSE: 1.4523545291338733
Random Forest RMSE: 0.47947562127127646
XGBoost RMSE: 0.4434667483009191
LightGBM RMSE: 0.4729026078070028
Neural Network RMSE: 0.7502811502385773

Linear Regression R2: -5.60415433597191
Random Forest R2: 0.43202495458589885
XGBoost R2: 0.4677063430034678
LightGBM R2: 0.5174024740259568
Neural Network R2: -0.7433654980645912


=== Station 305 ===
Linear Regression RMSE: 1.1645765110772852
Random Forest RMSE: 1.0386456382676683
XGBoost RMSE: 1.1060707602592261
LightGBM RMSE: 1.2591437629168016
Neural Network RMSE: 1.1430613713701858

Linear Regression R2: 0.03535718607322469
Random Forest R2: 0.26260160333982396
XGBoost R2: 0.13938134651117895
LightGBM R2: 0.2184771531399231
Neural Network R2: 0.07947169913624186


=== Station 325 ===
Linear Regression RMSE: 2.910769905701122
Random Forest RMSE: 1.0530980061718223
XGBoost RMSE: 0.8139665796807232
LightGBM RMSE: 0.8469181798027792
Neural Network RMSE: 1.3088656665016887

Linear Regression R2: -13.274895017920912
Random Forest R2: -0.809443026076218
XGBoost R2: -0.10700649646879934
LightGBM R2: -0.03542972119791754
Neural Network R2: -1.8444077848201084
